**Reading the Data**
* From csv file, and set the indexes

In [25]:

# import libraries
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,f1_score

import torch

import import_ipynb
from NNpythorch import *



In [ ]:
def plot_scores(X,y_test,criterion,weights ,threshold):
    
    y = np.ravel(y_test)
    
    for icolumn, column  in enumerate(X.columns):
        
        loss_test = criterion(torch.flatten(torch.tensor(X[column].values, dtype=torch.double)), torch.tensor(np.ravel(y_test.values)))
        
        y_pred = [1 if x >threshold else 0 for x in X[column].values]
        f1 = f1_score(y, y_pred,  sample_weight=weights)
        print('mode : {}, loss_test {}, f1_score {}'.format(column,loss_test,f1))

In [ ]:
def create_confusion_matrix(X,y, threshold):

    labels = [0,1]                
    
    fig, ax = plt.subplots(1)
    fig.set_size_inches(5, 5)

    cm = confusion_matrix(y,X>threshold , labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,  display_labels=labels)
    disp.plot(ax=ax)        
    ax.set_title('confusion matrix')

In [ ]:
def create_figure_distribution(X, column_y= []):    
    if (column_y == []):
        column_y=X.columns[-1]
    
    X= X.set_index([X.index.get_level_values(0),X.index.get_level_values(1),column_y])  
    data_stack = X.stack().to_frame()
    data_stack[column_y] = data_stack.index.get_level_values(2)
    data_stack.columns= [ 'value', column_y]
    data_stack['feature']=data_stack.index.get_level_values(3)
    ax=sns.displot(data_stack, x= 'value', hue =column_y,row='feature', common_norm=False, kind="kde", height=2, aspect=10)
    ax.set(ylim=(0, .5))

In [ ]:
def printFeatureImportance(classifier, indicators): 
    # features importance calculation and visualization
    
    importances = classifier.feature_importances_
    indices = np.argsort(importances)
    plt.figure()
    plt.set_size_inches(20, 4)
    plt.title('Feature Importances')
    plt.barh(range(len(indices)), importances[indices], color='b', align='center')
    plt.yticks(range(len(indices)), [indicators[i] for i in indices])
    plt.xlabel('Relative Importance')
    plt.show()

In [ ]:
def print_prob_events(y, y_predictions, title = ''): 

  
    fig1, axes1 = plt.subplots(1,2)
    fig1.tight_layout()
    fig1.set_size_inches(20, 4)
    time_axis = y.index.get_level_values(0)+y.index.get_level_values(1)/12

    color1 = 'red'
    color2 = 'blue'

    ax = axes1[0]    
    ax.scatter(time_axis, np.ravel (y_predictions), c=color1,marker = 'o', label='Predicted ')
    ax.scatter(time_axis, np.ravel (y), c=color2,marker = '*', label='Recorded' )
    ax.legend()        
    ax.set_title('Train - Damages VS time '+ title)
    ax.set_xlabel('Date')
    ax.set_ylabel('Number of damages')

    
    ax = axes1[1]
    ax.scatter(np.ravel (y), np.ravel (y_predictions), c=color1)
    ax.plot(np.ravel (y), np.ravel (y), c='blue')
    ax.set_title('Train - Damages vs predicted '+ title)
    ax.set_xlabel('Number of damages')
    ax.set_ylabel('Number of predicted damages')

    